In [1]:
print('Initializing data engineering!')
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import json, requests

Initializing data engineering!


### World data engineering
#### Fetching worldwide data

In [2]:
# df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')
# df = pd.read_json('https://covidapi.info/api/v1/country/BRA')
# df = pd.read_json('https://api.covid19api.com/dayone/country/brazil/status/confirmed')
#https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#cc76052f-6601-4645-80e5-ca7aaa36f8ef
df_countries = pd.read_csv('../data/world_countries_2019.csv')

df = pd.DataFrame()
url = "https://pomber.github.io/covid19/timeseries.json"
req = requests.get(url)
# r = r.json()
j = json.loads(req.text)

#### Fetching countries's pandemic data from Pomber's JSON to a dataframe 

In [3]:
# Loading countries names to dict
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Loading countries data do dict then to dataframe
dic = []
for country in countries:
    i = 0
    while i < len(j[country]):
        if j[country][i]['confirmed'] == 0:
            i += 1
            continue
        row = {'country': country, 'date': j[country][i]['date'], 'cases':j[country][i]['confirmed'],
               'deaths':j[country][i]['deaths'], 'recoveries':j[country][i]['recovered']}
        dic.append(row)
        i += 1 
df = pd.DataFrame.from_dict(dic)
df[df['country']=='Brazil'].tail()

,country,date,cases,deaths,recoveries
1906,Brazil,2020-5-23,347398,22013,142587
1907,Brazil,2020-5-24,363211,22666,149911
1908,Brazil,2020-5-25,374898,23473,153833
1909,Brazil,2020-5-26,391222,24512,158593
1910,Brazil,2020-5-27,411821,25598,166647


#### Feature engineering

In [4]:
for country in countries:
    qtdeDays = len(df[df.country == country])+1
    df.loc[df.country == country, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.country == country, 'case_day'] = df[df.country == country]['cases'].diff()    
    df.loc[df.country == country, 'death_day'] = df[df.country == country]['deaths'].diff()
    df.loc[df.country == country, 'recovery_day'] = df[df.country == country]['recoveries'].diff()

    # % daily variations
    df.loc[df.country == country, '%var_case_day'] = ((df[df.country == country]['case_day'] - df[df.country == country]['case_day'].shift()) / df[df.country == country]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_death_day'] = ((df[df.country == country]['death_day'] - df[df.country == country]['death_day'].shift()) / df[df.country == country]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_recovery_day'] = ((df[df.country == country]['recovery_day'] - df[df.country == country]['recovery_day'].shift()) / df[df.country == country]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.country == country) & (df.day == 1), 'case_day']= df.loc[(df.country == country) & (df.day==1), 'cases']
    df.loc[(df.country == country) & (df.day == 1), 'death_day']= df.loc[(df.country == country) & (df.day==1), 'deaths']
    df.loc[(df.country == country) & (df.day == 1), 'recovery_day']= df.loc[(df.country == country) & (df.day==1), 'recoveries']
    
    # Buscando a população do país e calculado os indicador per milhão
    if not df_countries[df_countries['country']==country].empty:
        million = df_countries[df_countries['country']==country]['PopTotal'].values[0] / 1000
        cases_million = (df[df.country == country]['cases'] / million).round(1)
        deaths_million = (df[df.country == country]['deaths'] / million).round(1)
        recoveries_million = (df[df.country == country]['recoveries'] / million).round(1)
        case_day_million = (df[df.country == country]['case_day'] / million).round(1)
        death_day_million = (df[df.country == country]['death_day'] / million).round(1)
        recovery_day_million = (df[df.country == country]['recovery_day'] / million).round(1)
        
    else:
        cases_million = 0
        deaths_million = 0
        recoveries_million = 0
        case_day_million = 0
        death_day_million = 0
        recovery_day_million = 0
    
    df.loc[df.country == country, 'cases_million'] = cases_million
    df.loc[df.country == country, 'deaths_million'] = deaths_million
    df.loc[df.country == country, 'recoveries_million'] = recoveries_million
    df.loc[df.country == country, 'case_day_million'] = case_day_million
    df.loc[df.country == country, 'death_day_million'] = death_day_million
    df.loc[df.country == country, 'recovery_day_million'] = recovery_day_million
    
    # moving averages (from last 7 days)
    df.loc[df.country == country, 'avg7_cases'] = df[df.country == country]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths'] = df[df.country == country]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries'] = df[df.country == country]['recovery_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    
    df.loc[df.country == country, 'avg7_case_day_million'] = df[df.country == country]['case_day_million'].rolling(window=7).mean().replace([np.nan], 0).round(3)
    df.loc[df.country == country, 'avg7_death_day_million'] = df[df.country == country]['death_day_million'].rolling(window=7).mean().replace([np.nan], 0).round(3)
    df.loc[df.country == country, 'avg7_recovery_day_million'] = df[df.country == country]['recovery_day_million'].rolling(window=7).mean().replace([np.nan], 0).round(3)
    

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recovery_day_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_case_day_million,avg7_death_day_million,avg7_recovery_day_million,perc_death,perc_recovery,active_cases
15815,Lesotho,2020-5-23,2,0,0,11,0,0,0,-100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
15816,Lesotho,2020-5-24,2,0,0,12,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
15817,Lesotho,2020-5-25,2,0,0,13,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
15818,Lesotho,2020-5-26,2,0,0,14,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
15819,Lesotho,2020-5-27,2,0,0,15,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [5]:
#Adjusting wrong negative variations (wrong number from the source)
# df.loc[df.case_day < 0, ['cases']] = df[df.case_day < 0].shift().cases#, ['cases']]
df.loc[df.case_day < 0, ['case_day']] = df[df.case_day < 0].shift().case_day#, ['cases']]
df.loc[df.cases_million < 0, ['cases_million']] = 0#df[df.cases_million < 0].cases_million.shift()#, ['cases']]


In [6]:
df.to_csv('../data/world_corona19_data.csv', index = False)

#### countries not located in UN dataset

In [7]:
# for country in countries:
#     if df_countries[df_countries['country']==country]['PopTotal'].empty:
#         print(country)


### Brazil data engineering

In [8]:
import requests

headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'x-parse-application-id':'unAFkcaNDeXajurGB7LChj8SgQYS2ptm',
}
req = requests.get("https://xx9p7hp1p7.execute-api.us-east-1.amazonaws.com/prod/PortalGeral",
                           headers=headers)
req_json = req.json()
url = req_json['results'][0]['arquivo']['url']
response = requests.get(url)
# response.content
with open('../data/gov_brazil_corona19_data.xlsx', 'wb') as f:
    f.write(response.content)

df = pd.read_excel('../data/gov_brazil_corona19_data.xlsx')
df.rename(columns={'regiao': 'region', 'estado':'state', 'data':'date', 'municipio':'city', 'casosAcumulado': 'cases', 'populacaoTCU2019':'population', 'obitosNovos':'death_day','obitosAcumulado':'deaths', 'Recuperadosnovos':'recovery_day'}, inplace= True)
df['date'] = df['date'].astype('datetime64[ns]')

df.recovery_day.fillna(0, inplace=True)
df.population.fillna(0, inplace=True)
df.codRegiaoSaude.fillna(0, inplace=True)
df.codmun.fillna(0, inplace=True)

df.drop(columns=['emAcompanhamentoNovos'], inplace=True)
df.fillna('-', inplace=True)
df.tail()

,region,state,city,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,date,semanaEpi,population,cases,casosNovos,deaths,death_day,recovery_day
122331,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-23,21,3015268.0,6251,303,95,5,0.0
122332,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-24,22,3015268.0,6638,387,104,9,0.0
122333,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-25,22,3015268.0,6930,292,114,10,0.0
122334,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-26,22,3015268.0,7210,280,124,10,0.0
122335,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-27,22,3015268.0,7761,551,133,9,0.0


#### Feature engineering

In [9]:
print('Iniciando feature engieering Brasil')
inicio = datetime.datetime.now()

states = df.state.unique()
# states = ['PE']
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for state in states:
    cities = df[df['state']==state].city.unique()
#     cities = ['-']
    for city in cities:
        indexes = (df['state']==state) & (df.city == city)
        qtdeDays = len(df[indexes])+1
        df.loc[indexes, 'day'] = (np.arange(1,qtdeDays,1))
    #     df.drop(df[case].index, inplace=True)
        # valores diários
        df.loc[indexes, 'case_day'] = df[indexes]['cases'].diff()    
        df.loc[indexes, 'death_day'] = df[indexes]['deaths'].diff()
        #acumulado
        df.loc[indexes, 'recoveries'] = df[indexes]['recovery_day'].sum()

        # % daily variations
        df.loc[indexes, '%var_case_day'] = ((df[indexes]['case_day'] - df[indexes]['case_day'].shift()) / df[indexes]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
        df.loc[indexes, '%var_death_day'] = ((df[indexes]['death_day'] - df[indexes]['death_day'].shift()) / df[indexes]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
        df.loc[indexes, '%var_recovery_day'] = ((df[indexes]['recovery_day'] - df[indexes]['recovery_day'].shift()) / df[indexes]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

        # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
        df.loc[(indexes) & (df.day == 1), 'case_day']= df.loc[(indexes) & (df.day==1), 'cases']
        df.loc[(indexes) & (df.day == 1), 'death_day']= df.loc[(indexes) & (df.day==1), 'deaths']

        # Buscando a população do estado/cidade e calculado os indicador per milhar
        if not df[indexes].population.empty:
            thousand = df[indexes]['population'] / 1000
            cases_thousand = (df[indexes]['cases'] / thousand).round(3)
            deaths_thousand = (df[indexes]['deaths'] / thousand).round(3)
            recoveries_thousand = (df[indexes]['recoveries'] / thousand).round(3)
            case_day_thousand = (df[indexes]['case_day'] / thousand).round(3)
            death_day_thousand = (df[indexes]['death_day'] / thousand).round(3)
            recovery_day_thousand = (df[indexes]['recovery_day'] / thousand).round(3)
        else:
            cases_thousand = 0
            deaths_thousand = 0
            recoveries_thousand = 0
            case_day_thousand = 0
            death_day_thousand = 0
            recovery_day_thousand = 0
    
        df.loc[indexes, 'cases_thousand'] = cases_thousand
        df.loc[indexes, 'deaths_thousand'] = deaths_thousand
        df.loc[indexes, 'recoveries_thousand'] = recoveries_thousand
        df.loc[indexes, 'case_day_thousand'] = case_day_thousand
        df.loc[indexes, 'death_day_thousand'] = death_day_thousand
        df.loc[indexes, 'recovery_day_thousand'] = recovery_day_thousand
        
        # moving averages (from last 7 days)
        df.loc[indexes, 'avg7_cases'] = df[indexes]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
        df.loc[indexes, 'avg7_deaths'] = df[indexes]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
        df.loc[indexes, 'avg7_recoveries'] = df[indexes]['recovery_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
        df.loc[indexes, 'perc_death'] = (df[indexes]['deaths']/df[indexes]['cases']*100).round(2) 
        df.loc[indexes, 'avg7_perc_death'] = df[indexes]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

        df.loc[indexes, 'avg7_case_day_thousand'] = df[indexes]['case_day_thousand'].rolling(window=7).mean().replace([np.nan], 0).round(3)
        df.loc[indexes, 'avg7_death_day_thousand'] = df[indexes]['death_day_thousand'].rolling(window=7).mean().replace([np.nan], 0).round(3)
        df.loc[indexes, 'avg7_recovery_day_thousand'] = df[indexes]['recovery_day_thousand'].rolling(window=7).mean().replace([np.nan], 0).round(3)
    
#         df.loc[indexes, 'perc_death'] = (df[indexes]['deaths']/df[indexes]['cases'] * 100).round(2)
#         df.loc[indexes, 'perc_recovery'] = (df[indexes]['recoveries']/df[indexes]['cases'] * 100).round(2)
#         df.loc[indexes, 'active_cases'] = df[indexes]['cases'] - df[indexes]['recoveries'] - df[indexes]['deaths']

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
df['recovery_day'] = df['recovery_day'].astype('int')

termino = datetime.datetime.now()
print('finalizado em ', termino-inicio)
df[indexes].tail()

Iniciando feature engieering Brasil
finalizado em  0:09:12.699462


,region,state,city,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,date,semanaEpi,population,...,avg7_cases,avg7_deaths,avg7_recoveries,perc_death,avg7_perc_death,avg7_case_day_thousand,avg7_death_day_thousand,avg7_recovery_day_thousand,perc_recovery,active_cases
122331,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-23,21,3015268.0,...,301.0,5.0,0.0,1.52,1.47,0.100,0.002,0.0,0.0,6156.0
122332,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-24,22,3015268.0,...,324.0,6.0,0.0,1.57,1.50,0.107,0.002,0.0,0.0,6534.0
122333,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-25,22,3015268.0,...,330.0,6.0,0.0,1.65,1.53,0.109,0.002,0.0,0.0,6816.0
122334,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-26,22,3015268.0,...,336.0,7.0,0.0,1.72,1.57,0.112,0.002,0.0,0.0,7086.0
122335,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,Brasília,2020-05-27,22,3015268.0,...,371.0,8.0,0.0,1.71,1.60,0.123,0.003,0.0,0.0,7628.0


In [10]:
df.to_csv('../data/brazil_corona19_data.csv', index = False)

In [11]:
# df[df.city=='Rio Claro'][['population','case_day','death_day','cases_thousand','deaths_thousand','active_cases']]

In [12]:
# df[df['country']=='Belgium']
print('Data engineering done!')

Data engineering done!
